# Partitioned Circuit Extraction Walkthrough

Final phase of the circuit partioning framework is the extraction of a partitioned quanutm circuit. 

Using the QuantumCircuitHyperGraph object and a node assignment function, we can infer the required teleportation operations and build a quantum cirucit.
The qubits are split across multiple registers - one for each partition. Each partition is also initialised with a communication qubit register and a classical bit register for facilitating the teleportation. Additional communication qubits may be added if there are not enough available.

## Build initial circuit

First build/import and transpile the circuit. We use a fixed depth random circuit as an example.

In [ ]:
from disqco.circuits.cp_fraction import cp_fraction
from disqco.circuits.QAOA import QAOA_random
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume

num_qubits = 6

circuit = cp_fraction(  num_qubits = num_qubits,
                        depth = 2*num_qubits,
                        fraction= 0.5,
                        seed=52345)

# circuit = QuantumVolume(num_qubits, 10)
# circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
# circuit = QFT(num_qubits, do_swaps=False)

num_qubits = circuit.num_qubits

circuit = transpile(circuit, basis_gates=['u', 'cp'])
depth = circuit.depth()


## Graph building

We then build the graph using the QuantumCircuitHyperGraph object. We also define the paremeteres of the QPU architecture using the qpu_info and comm_info variables, which tell us how many physical qubits are in each partition.

We can draw the resulting graph using matplotlib. 

In [ ]:
from disqco import QuantumCircuitHyperGraph, set_initial_partition_assignment

num_partitions = 2
qpu_size = int(num_qubits / num_partitions) + 1
qpu_info = [qpu_size] * num_partitions

comm_info = [1] * num_partitions

group_gates = True

graph = QuantumCircuitHyperGraph(circuit = circuit,
                                 group_gates = group_gates,
                                 anti_diag = True)


from disqco import QuantumNetwork

qnet = QuantumNetwork(qpu_info)

assignment = set_initial_partition_assignment(graph=graph,
                                              network=qnet)

from disqco.parti.FM.FM_methods import calculate_full_cost

cost = calculate_full_cost(graph, assignment, num_partitions)

print("E-bit cost for unoptimised assignment: ", cost)

In [ ]:
from disqco.drawing.tikz_drawing import *
from disqco.drawing.mpl_drawing import *

# %load_ext jupyter_tikz if using tikz drawing

graph.draw(dpi=300, output="mpl")  # or output="tikz"


## Naive extraction

We can first set a baseline for the distributed circuit by extracting a partitioned circuit directly from the unoptimised assignment function. This will likely result in a very deep circuit which does not use entanglement efficiently.

First import the extractor class, passing in the graph and the partition assignment that we initialised before.

In [ ]:
from disqco import PartitionedCircuitExtractor

extractor = PartitionedCircuitExtractor(graph=graph, 
                                        network=qnet,
                                         partition_assignment=assignment)



basic_partitioned_circuit = extractor.extract_partitioned_circuit()

basic_partitioned_circuit_EPR = transpile(basic_partitioned_circuit, basis_gates=['cp','u','EPR'])

print(f"Depth of partitioned circuit: {basic_partitioned_circuit.depth()}")

EPR_count = basic_partitioned_circuit_EPR.count_ops()['EPR']
print(f"EPR count in partitioned circuit: {EPR_count}")

We can draw the circuit to see where the teleportation blocks are occurring.

In [ ]:
if num_qubits < 12:
    fig = basic_partitioned_circuit.draw(output='mpl', style='bw', fold=100)
    display(fig)



To compare with original, we can transpile back into the U, CP gate-set, while choosing not to decompose the elementary entanglement generation options.

In [ ]:
decomposed_circuit = transpile(basic_partitioned_circuit, basis_gates = ['cp', 'u', 'EPR'])

print(f"Depth of decomposed partitioned circuit: {decomposed_circuit.depth()}")

We now optimise the node assignments using the recursive multilevel partitioning FM algorithm.

In [ ]:
from disqco.parti import FiducciaMattheyses

partitioner = FiducciaMattheyses( circuit=circuit,
                                  network=qnet,
                                  partition_assignment=assignment,
                                  hypergraph=graph)

results = partitioner.multilevel_partition()
final_cost = results['best_cost']
final_assignment = results['best_assignment']

print("Final e-bit cost:", final_cost)

graph.draw(network=qnet, assignment=final_assignment, output="mpl")


Now define a new extractor for with the optimised assignment. 

In [ ]:
from disqco import PartitionedCircuitExtractor

extractor_opt = PartitionedCircuitExtractor(graph = graph, 
                                            network= qnet,
                                            partition_assignment = final_assignment)

partitioned_circuit_opt = extractor_opt.extract_partitioned_circuit()


print(f"Depth of optimised partitioned circuit: {partitioned_circuit_opt.depth()}")

partitioned_circuit_opt_EPR = transpile(partitioned_circuit_opt, basis_gates = ['cp', 'u', 'EPR'])

if 'EPR' in partitioned_circuit_opt_EPR.count_ops():
    EPR_count_opt = partitioned_circuit_opt_EPR.count_ops()['EPR']
else:
    EPR_count_opt = 0

print(f"EPR count in optimised partitioned circuit: {EPR_count_opt}")




In [ ]:
decomposed_circuit_opt = transpile(partitioned_circuit_opt, basis_gates = ['cp', 'u', 'EPR'])
print(f"Depth of decomposed partitioned circuit: {decomposed_circuit_opt.depth()}")

if num_qubits < 12:
    fig = partitioned_circuit_opt.draw(output='mpl', style='bw', fold=100)
    display(fig)

The depth of the resulting circuit should significantly lower than the unoptimised variant, though of course still much larger than the un-partitioned circuit. Some additional overhead is unavoidable.

## Validation

We would like to validate the output given by the partitioned circuit, which we can do using qiskits sampler class.

First we need to add measurements to the original, unpartitioned circuit.

In [ ]:
from disqco.circuit_extraction.verification import run_sampler, plot, get_fidelity

circuit.measure_all()
data_circuit = run_sampler(circuit, shots=100000)
plot(data_circuit)


In [ ]:
data_partitioned_circuit = run_sampler(basic_partitioned_circuit, shots=100000)
plot(data_partitioned_circuit)


In [ ]:
data_partitioned_circuit_optimised = run_sampler(partitioned_circuit_opt, shots=100000)
plot(data_partitioned_circuit_optimised)

In [ ]:
fidelity = get_fidelity(data_circuit, data_partitioned_circuit, shots=100000)

print(f"Fidelity between original and unoptimised partitioned circuit: {fidelity}")

In [ ]:
fidelity = get_fidelity(data_circuit, data_partitioned_circuit_optimised, shots=100000)

print(f"Fidelity between original and optimised partitioned circuit: {fidelity}")